In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

plt.style.use('seaborn')

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

# 1. Understand the Problem (Business and Data)

## A. Understand the Business

Pada tahap ini akan dilakukan pemahaman terhadap permasalahan yang dihadapi. Pada kompetisi ini akan dilakukan pembuatan model yang dapat memprediksi apakah suatu penumpang dapat "survived" saat kejadian tenggelamnya kapal titanic. 

Problem Statement : Prediksi apakah suatu penumpang "survived" saat kejadian tenggelamnya kapal titanic.

Dengan membuat model yang dapat memprediksi sebuah penumpang "survived" atau tidak saat kejadian titanic maka perusahaan mungkin dapat melakukan tindakan preventif kedepannya berdasarkan karakteristik penumpang tersebut. 

*disclaimer: meskipun hal ini tidak dapat diterapkan dalam dunia nyata, tapi anggap saja manfaat pembuatan model ini seperti itu untuk kepentingan belajar*

## B. Understand the Data

Pada tahap ini akan dilakukan pemahaman terhadap dataset yang diberikan. Tahapan ini sangat penting karena pada kasus dilapangan terkadang memiliki kualitas data yang buruk sehingga perlu penanganan lebih lanjut.

Berikut data-data yang akan digunakan:
* train.csv - Data training yang berisi informasi terkait penumpang beserta informasi survived atau tidaknya penumpang tersebut
* test.csv - Data uji/test yang berisi informasi penumpang yang ingin diketahui apakah penumpang tersebut survived atau tidak
* sample_submission.csv - File yang berisi contoh format submisi pada platform kaggle

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train_copy = train.copy()
test_copy = test.copy()

## Exploratory Data Analysis (EDA)

Tahapan ini adalah tahapan dimana memahami data lebih jauh dan mencari tahu apa yang mungkin perlu diperhatikan atau menarik untuk disoroti kepada orang lain. Saat melakukan EDA, itu seperti mencari mutiara dalam kerang. Kita mungkin ingin membuka 100 kerang (test 100 hipotesis yang berbeda atau melihat data dengan 100 cara yang berbeda) untuk mencari mutiara mungkin 2 buah mutiara. Untuk melakukan EDA kita dapat melakukannya secara visualisasi, statistika deskriptif, ataupun melakukan uji hipotesis. 

### Check Missing Values

In [ ]:
missing = train_copy.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(fontsize=20)

In [ ]:
# train_copy.mean()
# train_copy.max()
# train_copy.min()
# train_copy.std()

# train.describe()

### 1 feature Analysis
Akan dilakukan visualisasi hanya 1 fitur saja.

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(x='Survived', data = train_copy)
plt.title("1 Feature")
plt.show()

In [ ]:
sns.boxplot(x=train_copy['Age'])

In [ ]:
sns.boxplot(x=train_copy['Fare'])

### 2 feature analysis
Akan dilakukan visualisasi dengan 2 fitur

In [ ]:
male_df = train_copy.loc[train_copy['Sex']=='male']
female_df = train_copy.loc[train_copy['Sex']=='female']

fig, axes = plt.subplots(1,2, figsize = (9,4))
sns.countplot(x = 'Survived', data = male_df, ax = axes[0])
sns.countplot(x = 'Survived',data = female_df, ax = axes[1])
plt.show()

In [ ]:
plt.figure(figsize = (6,4))
sns.countplot(x = 'Survived',data = train_copy, hue = 'Sex')
plt.title("Survival by Sex")
plt.show()

In [ ]:
sns.catplot(x = 'Survived', kind = 'count',col = 'Sex', data = train_copy, height = 3.3, aspect = 1.2)
plt.title("Survival by Sex")
plt.show()

In [ ]:
train_copy.corr()

In [ ]:
mask = np.triu(np.ones_like(train_copy.corr(), dtype=np.bool))

plt.figure(figsize=(16,6))
sns.heatmap(train_copy.corr(),mask=mask, annot=True, cmap='BrBG')

### 3 feature analysis
Akan dilakukan visualisasi dengan 3 fitur

In [ ]:
sns.set(font_scale=1)
plt.style.use('seaborn')

sns.catplot(x = 'Survived', kind = 'count',row = 'Sex',col = 'Embarked', data = train_copy,margin_titles = True, 
            height = 2.6, aspect = 1.2)
plt.show()

In [ ]:
n_rows = 2
n_cols = 2

fig, axes = plt.subplots(n_rows, n_cols,figsize=(12,8))

sns.countplot(x = 'Embarked',hue = 'Survived',data = train_copy,ax = axes[0][0])
sns.countplot(x = 'Pclass',hue = 'Survived',data = train_copy,ax = axes[0][1])
sns.countplot(x = 'SibSp',hue = 'Survived',data = train_copy,ax = axes[1][0])
sns.countplot(x = 'Parch',hue = 'Survived',data = train_copy,ax = axes[1][1])

axes[1][0].legend(loc = 'upper right')
axes[1][1].legend(loc = 'upper right')

plt.show()

## Survival by Pclass
Merupakan fitur penting karena menunjukan trend yang sangat sesuai dengan variabel target. Pclass adalah ukuran kelas ekonomi dari penumpang

- Pclass1 adalah first class
- Pclass2 adalah second class
- Pclass3 adalah third class

In [ ]:
sns.catplot(x = 'Pclass',y='Survived', kind = 'point',data = train_copy, hue = 'Sex',
           height = 4,aspect = 2)
plt.title("Pointplot : Survival vs Pclass")
plt.show()

- Dapat dilihat bahwa Pclass dan Survival rate berbanding terbalik, baik itu untuk laki-laki maupun perempuan
- Semakin tinggi status kelas ekonomi penumpang, maka semakin tinggi keselamatan penumpang dibandingkan yang lain, berlaku untuk laki2 dan perempuan

Tapi perlu dipertanyakan terlebih dahulu, berapa banyak penumpang yang berasal dari 1st class (Pclass) sebelum membuat asumsi.

In [ ]:
sns.catplot(x = 'Pclass', col = 'Survived', kind = 'count',data = train_copy)
plt.show()

Banyak yang tidak selamat pada 3rd Pclass, namun untuk penumpang yang selamat distribusi antar kelasnya mirip. 

Sehingga jika dilihat secara keseluruhan, survival rates menurun seiring kenaikan Pclass. Dengan demikian, kita dapat mengambil kesimpulan bahwa survival rates akan menurun jika seorang penumpang bukan merupakan status ekonomi kelas atas, terlepas dari laki2 maupun perempuan.

### Survival by Embarked


In [ ]:
sns.catplot(x = 'Embarked', col = 'Survived', kind = 'count', data = train_copy,
           height = 5, aspect = 1)
plt.show()

Distribusi penumpang menuju destinasi yang berbeda tidak merata.

- Surivival rates dan death rates mengikuti trend S>C>Q.
- Namun, dalam gambaran keseluruhan penumpang destinasi S merupakan penumpang terbanyak yang tidak selamat.
- Destinasi C dan Q hanya sedikit yang tidak selamat, meskipun hal ini kontribusi dari banyaknya penumpang yang menuju destinasi S
- Penumpang destinasi S merupakan penumpang terbanyak, dan survival ratesnya pun lebih tinggi meskipun death ratesnya pun tinggi.

In [ ]:
sns.catplot(x = 'Embarked',y = 'Survived',kind = 'point', hue = 'Sex',data = train_copy,height = 4, aspect = 2,)
plt.title("Survived vs Embarked by Sex")
plt.show()

- Penumpang destinasi Q, untuk pria memiliki survival rate yang rendah
- Penumpang destinasi S, untuk wanita memiliki survival rate yang rendah

Secara keseluruhan, wanita lebih memiliki survival rate yang tinggi dibandingkan pria terpelas dari tujuan keberangkatan

### Survival by Embarked and Pclass

In [ ]:
sns.catplot(x='Embarked', kind = 'count', col = 'Pclass', data=train_copy)
plt.show()

- Dapat dilihat bahwa kebanyakan penumpang akan berangkat menuju S dan merupakan penumpang kelas 3rd Pclass. Dan juga, dapat dilihat total jumlah penumpang tidak terdistribusi secara merata untuk setiap destinasi.
- Orang yang menuju S disetiap Pclass memiliki jumlah penumpang terbanyak.
- Karna penumpang yang menuju Q sangatlah sedikit pada Pclass 1 dan Pclass 2 (Pclass dengan survival rates yg tinggi), "FUNGSI SURVIVAL" terhadap destinasi Q cenderung memiliki penyimpangan yang tinggi,karena jumlah penumpang sangat sedikit, maka perubahan keadaan 1 penumpang akan menggangu fungsi tersebut.

In [ ]:
sns.catplot(x = 'Embarked', kind = 'point',y = 'Survived',col = 'Pclass', data = train_copy,margin_titles = True)
plt.show()

Dapat dilihat bahwa Garis (Confidence Interval) pada destinasi Q baik untuk Pclass 1 dan Pclass 2 sangatlah panjang.

In [ ]:
sns.catplot(x = 'Embarked',y = 'Survived',kind = 'point',col = 'Pclass', hue = 'Sex', data = train_copy)
plt.show()

Trend yang sangat aneh terdektesi, yaitu pria yang menuju Q memiliki survival rates yang rendah. Dan juga dapat disimpulkan keanehan grafik point plot sebelumnya terjadi karena Pclass 1 dan Pclass 2 merupakan penyebabnya karena, semua pria tidak selamat dan semua perempuan selamat untuk destinasi Q.

- Survival rates wanita paling rendah ketika wanita tersebut merupakan 3rd Pclass dan menuju ke S. 

### Survival by Age and Pclass
Karena age merupakan variabel continuous, maka diperlukan teknik visualisasi yang lain. Teknik yang akan digunakan adalah swarmplots

In [ ]:
fig, axes = plt.subplots(figsize = (13,7))
sns.swarmplot(y = 'Age', x = 'Pclass', hue = 'Survived',dodge = True,data = train_copy, ax = axes)
plt.title("Swarmplot : Age vs Survival by Pclass")
plt.show()

In [ ]:
sns.catplot(kind = 'swarm',x = 'Pclass', y = 'Age', col = 'Sex', hue = 'Survived',dodge = True,data = train_copy)
plt.show()

Dapat dilihat bahwa dibawah umur 10 tahun, banyak titik hijau yang merupakan kelas Pclass1 dan Pclass2. Namun untuk tingkat keselamatan anak2 Pclass3 tidak terlalu bagus. 

Dan juga dikarenakan jumlah wanita yang meninggal pada Pclass1 hanya berjumlah 3 orang, dan 2 dari ketiga wanita tersebut saling tumpang tindih dengan daerah yang selamat, maka dapat diasumsikan bahwa semua wanita yang merupakan Pclass1 akan memiliki survival rates tinggi terlepas dari umur berapapun. Dan hal ini juga berlaku untuk wanita yang merupakan Pclass2.

### Survival by Fare and Pclass

In [ ]:
fig, axes = plt.subplots(figsize = (13,7))
sns.swarmplot(x = 'Pclass', y = 'Fare', hue = 'Survived',dodge = True,data = train_copy, ax = axes)
plt.show()

Dapat dilihat penumpang yang membayar diatas 500 (3 orang) selamat dalam kejadian titanic.

Namun, untuk sisanya terdistribusi secara merata dan tidak ada perbedaan diantara mereka. Orang yang membayar dengan harga tinggi tidak selamat begitu juga dengan orang yang membayar dengan rendah, disemua Pclass.

Mari lihat lebih dalam lagi dengan memisahkannya menurut gender.

In [ ]:
sns.catplot(kind = 'swarm',x = 'Pclass', y = 'Fare', col = 'Sex', hue = 'Survived',dodge = True,data = train_copy)
plt.show()

Sangat menarik ! Bahwa penumpang pria yang membayar lebih mahal pada Pclass1 dan Pclass2 semuanya tidak selamat!

Untuk wanita yang merupakan Pclass1 dan Pclass2 kebanyakan selamat! Dan pada Pclass3 untuk wanita yang membayar lebih mahal tidak selamat (3 orang)

Dikarenakan Age dan Fare merupakan variabel continous, maka untuk mendeteksi pola akan dilakukan binning,jika tidak akan menyebabkan overfitting. Binning akan dilakukan di Feature Engineering, yaitu membuat kolom baru berdasarkan kolom-kolom yang ada.

## Preprocessing
Sebelum melakukan FE, akan dilakukan praproses terlebih dahulu

In [ ]:
train_copy.isna().sum()

In [ ]:
test_copy.isna().sum()

In [ ]:
train_copy['Age'] = train_copy['Age'].fillna(train_copy['Age'].mean())
test_copy['Age'] = test_copy['Age'].fillna(test_copy['Age'].mean())

In [ ]:
train_copy['Cabin'] = train_copy['Cabin'].fillna('No Cabin')
test_copy['Cabin'] = test_copy['Cabin'].fillna('No Cabin')

In [ ]:
test_copy['Fare'] = test_copy['Fare'].fillna(test_copy['Fare'].mean())
train_copy.head()

In [ ]:
train_copy.loc[train_copy['Sex'] == 'female', 'Sex'] = 1
train_copy.loc[train_copy['Sex'] == 'male', 'Sex'] = 0

test_copy.loc[test_copy['Sex'] == 'female', 'Sex'] = 1
test_copy.loc[test_copy['Sex'] == 'male', 'Sex'] = 0

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_copy['Cabin'])
train_copy['Cabin'] = le.transform(train_copy['Cabin'])


le_test = preprocessing.LabelEncoder()
le_test.fit(test_copy['Cabin'])
test_copy['Cabin'] = le_test.transform(test_copy['Cabin'])

In [ ]:
train_copy.head()

In [ ]:
test_copy.head()

## Feature Engineering

In [ ]:
for df in [train_copy, test_copy]:
    
    df['FamilySize'] = df['Parch']+df['SibSp']+1
    
    df['IsAlone']=0
    df.loc[(df.FamilySize==1),'IsAlone'] = 1

In [ ]:
train_copy.head()

In [ ]:
test_copy.head()

### Family size as Survival Function

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (15,6))

sns.countplot(x = 'FamilySize', data = train_copy, ax = axes [0])
sns.barplot(x = 'FamilySize', y = 'Survived', data = train_copy, ax = axes[1])
plt.show()

- Penumpang titanic kebanyakan berangkat hanya dengan dirinya sendiri dan survival rate mereka rendah (close to 30%) dengan deviasi yang kecil.
- Penumpang dengan Family Size 5,6, dan 7 juga memiliki survival rate sekitar 30%, tapi karena jumlah datanya sedikit maka garis panjang ditengah visualisasi menunjukkan deviasi yang besar.
- Dan dapat diambil kesimpulan, bahwa penumpang yang sendiri ketika berada di titanic bukan merupakan ide yang bagus untuk survival.


### IsAlone and Sex

In [ ]:
fig, saxis = plt.subplots(1, 2,figsize=(12,5))

sns.pointplot(x = 'IsAlone', y = 'Survived',  kind = 'point', data = train_copy, ax = saxis[0])
sns.pointplot(x = 'IsAlone', y = 'Survived', hue = 'Sex', kind = 'point', data = train_copy, ax = saxis[1])
plt.show()

Secara gambaran besar, dapat disimpulkan bahwa penumpang yang tidak sendiri memiliki survival rates yang tinggi.

Namun, hal ini tidak berlaku jika dilihat berdasarkan gender. Wanita lebih cenderung selamat ketika sendiri dibandingkan pria.
- Wanita yang sendiri, memiliki survival rates yang tinggi (close to 80%) dibandingkan wanita yg tidak sendiri (close to 70%).
- Namun, secara keseluruhan survival chances yang dimiliki wanita tetap sangat lebih tinggi dibandingkan pria, terlepas dari apakah dia sendiri atau tidak.

### One Hot Encoding Embarked

In [ ]:
train_copy = pd.get_dummies(train_copy,columns = ['Embarked'],drop_first = False)
test_copy = pd.get_dummies(test_copy, columns = ['Embarked'],drop_first = False)
train_copy.head()

### Binning Age

In [ ]:
train_copy.loc[train_copy['Age'] < 10, 'Age'] = 1
train_copy.loc[(train_copy['Age']>= 10) & (train_copy['Age']<= 20),'Age'] = 2
train_copy.loc[(train_copy['Age']> 20) & (train_copy['Age']<= 40), 'Age'] = 3
train_copy.loc[(train_copy['Age']> 40) & (train_copy['Age']<= 60),'Age']= 4
train_copy.loc[(train_copy['Age']>= 60), 'Age']= 5

In [ ]:
test_copy.loc[test_copy['Age'] < 10, 'Age'] = 1
test_copy.loc[(test_copy['Age']>= 10) & (test_copy['Age']<= 20),'Age'] = 2
test_copy.loc[(test_copy['Age']> 20) & (test_copy['Age']<= 40), 'Age'] = 3
test_copy.loc[(test_copy['Age']> 40) & (test_copy['Age']<= 60),'Age']= 4
test_copy.loc[(test_copy['Age']>= 60), 'Age']= 5

In [ ]:
train_copy.head()

In [ ]:
test_copy.head()

## Modelling
Machine learning model sangatlah banyak, namun kita akan mengawali pemodelan dengan logistic regression dan dilanjutkan dengan beberapa model lainnya

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
logreg = LogisticRegression()
svc = SVC()
ext = ExtraTreesClassifier()
cat = CatBoostClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

## Evaluasi

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_copy.head()

In [ ]:
X = train_copy[['Pclass','Age','SibSp','Parch','Fare']]
y = train_copy['Survived']

final_test = test_copy[['Pclass','Age','SibSp','Parch','Fare']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = logreg.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

## Prediction

In [ ]:
model = logreg.fit(X,y)
pred_submit = model.predict(final_test)

In [ ]:
sample = pd.read_csv('../input/titanic/gender_submission.csv')
sample.head()

In [ ]:
sample.Survived.value_counts()

In [ ]:
sample['Survived'] = pred_submit
sample.Survived.value_counts()

In [ ]:
sample.to_csv("first_submit_base.csv", index=False)